In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import numpy as np
import numpy.linalg as la
from scipy.integrate import RK45

import sympy as sym
# Heaviside numerics workaround
from sympy.utilities.lambdify import implemented_function
lambdafied_heaviside = implemented_function('H', lambda x: np.heaviside(x,0))

from table_maker import *
from functools import partial
from itertools import *
from math import ceil

import pickle

def cos_bell(x, center=0, width=2*np.pi, height=1):
    return (np.cos((x-center)/width*2*np.pi)+1)/2*height * np.heaviside(x-center+width/2,0) * np.heaviside(-x+center+width/2,0)

In [2]:
import matplotlib
matplotlib.rcParams.update({'font.size': 24})

From [Kilpatrick and Bressloff 2010](https://doi.org/10.1016/j.physd.2009.06.003)
$$\begin{align*}
    \mu u_t &= -u + \int_{-\infty}^\infty w(x,x^\prime) q(x^\prime,t) f( u(x^\prime,t) - a(x^\prime,t)) \ dx^\prime \\
    q_t &= \frac{1 - q}{\alpha} - \beta q f(u - a) \\
    \epsilon a_t &= -a + \gamma f(u - a)
\end{align*}$$

Modified version - remove synaptic depression $q$
$$\begin{align*}
    \mu u_t &= -u + \int_{-\infty}^\infty w(x,x^\prime) f( u(x^\prime,t) - a(x^\prime,t)) \ dx^\prime \\
    \alpha a_t &= -a + \gamma f(u - a)
\end{align*}$$
(note that parameters have been relabeled)

Beginning at (Kilpatrick & Bressloff 2010)[https://www.sciencedirect.com/science/article/abs/pii/S0167278909001833] equations 3.30 and 3.31. We will use the same notation they use (i.e. we will use the $\alpha$ and $\epsilon$ they use and substitute back to our notation later. Additionally, sympy uses $\theta$ to denote the Heaviside function. We will thus use $\tau$ to denote the threshold here.

In [3]:
import collections.abc
def collect_eqn(expr, terms):
    if not isinstance(terms, collections.abc.Container):
        term = terms # isn't a list or tuple
        left, right = 0,0
        for my_expr in expr.args:
            has_term = False
            for my_term in sym.postorder_traversal(my_expr):
                if term == my_term:
                    has_term = True
                    break
            if has_term:
                left += my_expr
            else:
                right -= my_expr
        return sym.Eq(left, right, evaluate=False)
    else: # isn't a list or tuple
        left_total, right_total = 0, expr
        for term in terms:
            left, right = collect_eqn(right_total, term).args
            left_total += left
            right_total = -right
        return sym.Eq(left_total, right_total, evaluate=False)
    
def my_print(expr):
    global eviron_print_tex
    if eviron_print_tex:
        print('$$' + sym.latex(expr) + '$$')
    else:
        display(expr)
        
def batch_print(expr_list, punctuation='.'):
    global eviron_print_tex
    if eviron_print_tex:
        print_str = '$$\\begin{align*}\n'
        for expr in expr_list:
            print_str += '\t' + sym.latex(expr).replace('=','&=') + '\\\\\n'
        # remove last new-line and add ending puncuation
        print_str = print_str[:-3] + punctuation + '\n'
        print_str += '\\end{align*}$$'
        print(print_str)
    else:
        for expr in expr_list:
            display(expr)
            
def py_code(expr, replace_numpy=True, replace_unicode=True, numpy_package='np.'):
    code = sym.ccode(expr)
    
    if replace_unicode:
        unicode_replacements = [
            ('\\alpha', 'α'),
            ('\\beta', 'β'),
            ('\\gamma', 'γ'),
            ('\\Gamma', 'Γ'),
            ('\\delta', 'δ'),
            ('\\Delta', 'Δ'),
            ('\\epsilon', 'ϵ'),
            ('\\zeta', 'ζ'),
            ('\\eta', 'η'),
            ('\\theta', 'θ'),
            
        ]
        for unicode, symbol in unicode_replacements:
            code = code.replace(unicode, symbol)
    
    if replace_numpy:
        numpy_replacements = [
            ('exp', numpy_package + 'exp'),
            ('log', numpy_package + 'log'),
            ('pow', '( lambda base, exponent: base**exponent )')
        ]
        for c_func, np_func in numpy_replacements:
            code = code.replace(c_func, np_func)
        
    return code

In [6]:
eviron_print_tex = False

μ, α, γ, τ, c, Δ = sym.symbols('\\mu \\alpha \\gamma, \\tau, c \\Delta', real=True, positive = True)
x, xp, t, ξ, ξp = sym.symbols('x x^\\prime t \\xi \\xi^\\prime', real=True)
u = sym.Function('u', real=True)(x,t)
a = sym.Function('u', real=True)(x,t)
U = sym.Function('U', real=True)(ξ)
A = sym.Function('A', real=True)(ξ)

G = sym.integrate( sym.Heaviside(ξp-ξ)*sym.exp(ξ-ξp) + sym.Heaviside(ξ-ξp)*sym.exp(ξp-ξ), (ξp, -Δ, 0) )/2
eqn1 = μ*c*U.diff(ξ) - U + G
eqn2 = α*c*A.diff(ξ) - A + γ*( sym.Heaviside(ξ+Δ) - sym.Heaviside(ξ) )


print('Evaluating the integral, we then obtain')
batch_print( [sym.Eq(0, eqn1), sym.Eq(0, eqn2)] )

print('We will solve each system using Laplace transforms')
s = sym.symbols('s', real=True, positive=True)
Leqn1, eqn1_domain, _ = sym.laplace_transform(eqn1, ξ, s)
Leqn2, eqn2_domain, _ = sym.laplace_transform(eqn2, ξ, s)
# manually apply L(U') = s*L(U) - U(0)
LUsym = sym.laplace_transform(U, ξ, s)
Leqn1 = Leqn1.subs(sym.laplace_transform(U.diff(ξ), ξ, s), s*LUsym - U.subs(ξ,0))
LAsym = sym.laplace_transform(A, ξ, s)
Leqn2 = Leqn2.subs(sym.laplace_transform(A.diff(ξ), ξ, s), s*LAsym - A.subs(ξ,0))

batch_print( [sym.Eq(0, Leqn1), sym.Eq(0, Leqn2)], punctuation='')
print('which are jointly true on the domain $s>%s$.' % sym.latex(sym.Max(eqn1_domain, eqn2_domain)) )

print('Solving for the Laplace transform variables we have')

LU_sub = sym.solve(Leqn1, LUsym)[0]
LA_sub = sym.solve(Leqn2, LAsym)[0]

batch_print( [sym.Eq(LUsym, LU_sub), sym.Eq(LAsym, LA_sub)], punctuation='')

print('Applying the inverse Laplace transform then gives us the solutions for $\\xi > 0$')

U_sub = sym.inverse_laplace_transform(LU_sub, s, ξ)
A_sub = sym.inverse_laplace_transform(LA_sub, s, ξ)

batch_print( [sym.Eq(U, U_sub), sym.Eq(A, A_sub)], punctuation='')
print('and thus')
batch_print([sym.Eq( U-A, (U_sub - A_sub).collect(sym.exp(ξ/μ/c)) )])

print('The right boundary condition then gives')
cond1 = sym.limit(U_sub.subs( U.subs(ξ, 0), τ ) - A_sub.subs( A.subs(ξ,0), 0) , ξ, sym.oo).args[1].args[0]

batch_print([sym.Eq(0, A.subs(ξ,0)), sym.Eq(0, cond1)])

print('Or equivalently,')
Δ_sub = sym.solve(cond1, Δ)[0]
c_sub = sym.solve(cond1, c)[0]

batch_print([sym.Eq(c, c_sub), sym.Eq(Δ, Δ_sub)])


print('To get solutions for $\\xi < 0$, we will make the substitution $\\zeta = -\\xi $ and use the variables V(\\zeta) = U(\\xi), and B(\\zeta) = A(\\xi).')
print('Our equations become')

ζ = sym.symbols('\\zeta', real=True)
V = sym.Function('V')(ζ)
B = sym.Function('B')(ζ)
eqn1V = eqn1.subs(U.diff(ξ), -V.diff(ζ)).subs(U, V).subs(ξ, -ζ)
eqn2B = eqn2.subs(A.diff(ξ), -B.diff(ζ)).subs(A, B).subs(ξ, -ζ)

batch_print([sym.Eq(0, eqn1V), sym.Eq(0, eqn2B)])

print('We take the Laplace transform with respect to $\\zeta$ now, to obtain')

Leqn1V, eqn1V_domain, _ = sym.laplace_transform(eqn1V, ζ, s)
LVsym = sym.laplace_transform(V, ζ, s)
Leqn1V = Leqn1V.subs(sym.laplace_transform(V.diff(ζ), ζ, s), s*LVsym - V.subs(ζ,0))

Leqn2B, eqn2B_domain, _ = sym.laplace_transform(eqn2B, ζ, s)
LBsym = sym.laplace_transform(B, ζ, s)
Leqn2B = Leqn2B.subs(sym.laplace_transform(B.diff(ζ), ζ, s), s*LBsym - B.subs(ζ,0))

batch_print([sym.Eq(0, Leqn1V), sym.Eq(0, Leqn2B)])

print('Next, we solve for the transformed varialbes')

LV_sub = sym.solve(Leqn1V, LVsym)[0] 
LB_sub = sym.solve(Leqn2B, LBsym)[0] 

batch_print([sym.Eq(LVsym, LV_sub), sym.Eq(LBsym, LB_sub)], punctuation='')

print('and apply an inverse transform')

#need to manually partial fraction decomp - sympy is struggling
coeff = -(μ*c*s + 1)
LV_sub_rational1 = 1/(s-1)/2
LV_sub_rational2 = 1/s/(s+1)/2
expr = LV_sub_rational1*(1 - sym.exp(-Δ)) + LV_sub_rational2 * (1 - sym.exp(-Δ*s)) + μ*c*V.subs(ζ,0)
expr += coeff * LVsym
assert expr.expand() - Leqn1V.expand() == 0 # properly factored expression
A = -1/(1+c*μ)/2
B = A + sym.Rational(1,2)
LV_sub_pfd1 = A/(s-1) + B/(1+c*μ*s)
assert (LV_sub_pfd1 - LV_sub_rational1/coeff).simplify() == 0
A = -sym.Rational(1,2)
B = -A/(1-μ*c)
D = -B - A*(1+μ*c)
LV_sub_pfd2 = A/s + B/(s+1) + D/(1+μ*c*s)
assert (LV_sub_pfd2 - LV_sub_rational2/coeff).simplify() == 0
LV_sub = -(LV_sub_pfd1*(1-sym.exp(-Δ)) + LV_sub_pfd2*(1-sym.exp(-Δ*s)) + μ*c*V.subs(ζ,0)/coeff)
assert Leqn1V.subs(LVsym, LV_sub).simplify() == 0 # partial fraction decomposition satisfies the condition
# sympy refuses to find the inverse laplace transform of the fourth term in this sum
terms = [-LV_sub_pfd1, LV_sub_pfd1*sym.exp(-Δ), - LV_sub_pfd2,  LV_sub_pfd2*sym.exp(-Δ*s), - μ*c*V.subs(ζ,0)/coeff]
term3 = sum([arg*sym.exp(-Δ*s) for arg in terms[3].args[0].args])
# LV_sub = sum(terms)
# assert Leqn1V.subs(LVsym, LV_sub).simplify() == 0
# test = sum(term for i,term in enumerate(terms) if i!=3) + term3
# assert (LV_sub - test).simplify() == 0 

new_terms = [term for i,term in enumerate(terms) if i!=3] + [term3]
assert (sum(new_terms) - sum(terms)).simplify() == 0 # we have refactored the expression correctly
V_sub = sum(sym.inverse_laplace_transform(term, s, ζ) for term in new_terms)





Evaluating the integral, we then obtain


Eq(0, \mu*c*Derivative(U(\xi), \xi) + (1 - exp(\xi))*Heaviside(-\xi)/2 - (1 - exp(\Delta + \xi))*Heaviside(-\Delta - \xi)/2 - U(\xi) - exp(-\Delta - \xi)/2 + exp(-\xi)/2)

Eq(0, \alpha*c*Derivative(A(\xi), \xi) + \gamma*(-Heaviside(\xi) + Heaviside(\Delta + \xi)) - A(\xi))

We will solve each system using Laplace transforms


Eq(0, \mu*c*(s*LaplaceTransform(U(\xi), \xi, s) - U(0)) - LaplaceTransform(U(\xi), \xi, s) + 1/(2*(s + 1)) - exp(-\Delta)/(2*(s + 1)))

...

Eq(0, -\alpha*c*(s*LaplaceTransform(B(\zeta), \zeta, s) - B(0)) + \gamma/s - \gamma*exp(-\Delta*s)/s - LaplaceTransform(B(\zeta), \zeta, s))

Next, we solve for the transformed varialbes


Eq(LaplaceTransform(V(\zeta), \zeta, s), (2*\mu*c*s**3*V(0)*exp(\Delta*s + \Delta) - 2*\mu*c*s*V(0)*exp(\Delta*s + \Delta) - s**2*exp(\Delta*s) + s**2*exp(\Delta*s + \Delta) - s*exp(\Delta) - s*exp(\Delta*s) + 2*s*exp(\Delta*s + \Delta) + exp(\Delta) - exp(\Delta*s + \Delta))*exp(-\Delta*s - \Delta)/(2*s*(\mu*c*s**3 - \mu*c*s + s**2 - 1)))

Eq(LaplaceTransform(B(\zeta), \zeta, s), (\alpha*c*s*B(0)*exp(\Delta*s) + \gamma*exp(\Delta*s) - \gamma)*exp(-\Delta*s)/(s*(\alpha*c*s + 1)))

and apply an inverse transform


In [9]:
sym.inverse_laplace_transform(LB_sub, s, ζ)

-\gamma*InverseLaplaceTransform(1/(\alpha*c*s**2*exp(\Delta*s) + s*exp(\Delta*s)), s, \zeta, _None) + \gamma*Heaviside(\zeta) - \gamma*exp(-\zeta/(\alpha*c))*Heaviside(\zeta) + B(0)*exp(-\zeta/(\alpha*c))*Heaviside(\zeta)

In [12]:
LB_sub.simplify()

(\alpha*c*s*B(0)*exp(\Delta*s) + \gamma*exp(\Delta*s) - \gamma)*exp(-\Delta*s)/(s*(\alpha*c*s + 1))

In [24]:
test = LB_sub.args[0] * LB_sub.args[1] * (LB_sub.args[2] * LB_sub.args[3]).expand()
assert (test - LB_sub).simplify() == 0

In [26]:
sym.inverse_laplace_transform(test, s, ζ)

-\gamma*InverseLaplaceTransform(1/(\alpha*c*s**2*exp(\Delta*s) + s*exp(\Delta*s)), s, \zeta, _None) + \gamma*Heaviside(\zeta) - \gamma*exp(-\zeta/(\alpha*c))*Heaviside(\zeta) + B(0)*exp(-\zeta/(\alpha*c))*Heaviside(\zeta)

In [27]:
test

(\alpha*c*s*B(0) + \gamma - \gamma*exp(-\Delta*s))/(s*(\alpha*c*s + 1))

In [ ]:
# do a manual pdf again =(